In [2]:
# Librerias
import pandas as pd
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import PowerTransformer

# Archivo Dataset Final
df = pd.read_excel("/content/Final_DF.xlsx")

print("Dimensión del dataset:", df.shape)
df.head()

Dimensión del dataset: (168, 524)


,id,visita,grupo,periodo,tiempo,tratamiento,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,...,pgh_por_eficiencia,pgh_eficiencia,pgh_perturbaciones,pgh_disfuncion_dia,pgh_duracion_sueño,pgh_total,cat_pgh_total,total_meds,menst_flag,menst_tipo
0,1,1,1,1,1,1,2,5,29,4,...,80.00000,1,1,0,2,5,1,2,0,MPM
1,1,2,1,1,2,1,2,5,29,4,...,104.00000,0,1,0,0,1,1,0,0,MPM
2,1,3,1,2,1,2,2,5,29,4,...,96.55173,0,1,1,1,4,1,0,0,MPM
3,1,4,1,2,2,2,2,5,29,4,...,103.57140,0,1,0,0,1,1,0,0,MPM
4,2,1,2,1,1,2,2,2,21,4,...,85.71429,0,1,1,2,4,1,5,0,Hombre


In [16]:
# Creando columnas identificadoras
df['Treatment'] = np.where(df['tratamiento'] == 1, 'Intervencion', 'Control')
df['Time'] = np.where(df['tiempo'] == 1, 'Pre', 'Post')

df[['Treatment', 'Time']].head()

,Treatment,Time
0,Intervencion,Pre
1,Intervencion,Post
2,Control,Pre
3,Control,Post
4,Control,Pre


In [17]:
df = df.drop(['visita', 'periodo', 'tiempo', 'tratamiento'], axis='columns')

print("Dataset actualizado:", df.shape)

Dataset actualizado: (168, 522)


In [18]:
num_df = df.select_dtypes(include=["int64", "float64"])

# Excluir ID si existe (no es variable clínica)
num_df = num_df.drop(columns=["ID"], errors="ignore")

print("Variables numéricas evaluadas:", num_df.shape)


Variables numéricas evaluadas: (168, 506)


In [19]:
def detectar_outliers_std(df_numeric, k=3):
    resultados = []

    for col in df_numeric.columns:
        serie = df_numeric[col].dropna()

        mean = serie.mean()
        std = serie.std()

        if std == 0:
            continue

        outliers = serie[(serie < mean - k*std) | (serie > mean + k*std)]

        resultados.append({
            "variable": col,
            "num_outliers": len(outliers)
        })

    resultados_df = pd.DataFrame(resultados)
    return resultados_df.sort_values("num_outliers", ascending=False)

In [20]:
for k in [3, 4, 5, 6]:
    temp = detectar_outliers_std(num_df, k)
    total_vars = (temp["num_outliers"] > 0).sum()
    print(f"Umbral ±{k} STD → Variables con outliers: {total_vars}")

Umbral ±3 STD → Variables con outliers: 295
Umbral ±4 STD → Variables con outliers: 161
Umbral ±5 STD → Variables con outliers: 83
Umbral ±6 STD → Variables con outliers: 48


Se aplicó un método basado en desviación estándar, utilizando umbrales de ±3 y ±4 desviaciones estándar.

Este enfoque permite identificar observaciones extremadamente alejadas de la media bajo el supuesto de distribución aproximadamente normal. Sin embargo, debido a la presencia de variables ordinales y escalas codificadas numéricamente, los resultados se interpretan de manera descriptiva y no necesariamente como errores en los datos.

In [21]:
tabla_std = pd.DataFrame({
    "Método": ["STD (Z-score)"]*4,
    "Umbral": ["±3 STD", "±4 STD", "±5 STD", "±6 STD"],
    "Variables con outliers": [295, 161, 83, 48],
    "Interpretación": [
        "Detección estándar, identifica valores moderadamente extremos",
        "Más conservador, resalta valores claramente atípicos",
        "Muy conservador, solo valores altamente extremos",
        "Extremadamente conservador, únicamente casos raros"
    ]
})

tabla_std

,Método,Umbral,Variables con outliers,Interpretación
0,STD (Z-score),±3 STD,295,"Detección estándar, identifica valores moderad..."
1,STD (Z-score),±4 STD,161,"Más conservador, resalta valores claramente at..."
2,STD (Z-score),±5 STD,83,"Muy conservador, solo valores altamente extremos"
3,STD (Z-score),±6 STD,48,"Extremadamente conservador, únicamente casos r..."


In [22]:
for (treat, time), subdf in df.groupby(["Treatment", "Time"]):

    print("\n==============================")
    print("Grupo:", treat, "| Tiempo:", time)

    sub_num = subdf.select_dtypes(include=["int64", "float64"])
    sub_num = sub_num.drop(columns=["ID"], errors="ignore")

    temp = detectar_outliers_std(sub_num, k=3)
    total_vars = (temp["num_outliers"] > 0).sum()

    print("Variables con outliers (±3 STD):", total_vars)


Grupo: Control | Tiempo: Post
Variables con outliers (±3 STD): 212

Grupo: Control | Tiempo: Pre
Variables con outliers (±3 STD): 215

Grupo: Intervencion | Tiempo: Post
Variables con outliers (±3 STD): 192

Grupo: Intervencion | Tiempo: Pre
Variables con outliers (±3 STD): 214


El análisis de valores atípicos mediante el criterio basado en desviación estándar (Z-score) mostró que el número de variables con observaciones extremas disminuye conforme se incrementa el umbral utilizado.

Con ±3 desviaciones estándar se identificaron 295 variables con posibles outliers, mientras que con criterios más conservadores como ±6 desviaciones estándar únicamente 48 variables presentaron valores extremadamente alejados de la media.

Estos resultados se interpretan de forma descriptiva, considerando que parte del conjunto de datos incluye escalas ordinales y variables codificadas numéricamente, donde la presencia de valores extremos no necesariamente implica errores, sino variabilidad inherente al estudio.

# Transformación Yeo–Johnson

In [3]:
num_df = df.select_dtypes(include=["int64", "float64"])
num_df = num_df.drop(columns=["ID"], errors="ignore")

print("Variables numéricas:", num_df.shape)

Variables numéricas: (168, 510)


In [5]:
pt = PowerTransformer(method="yeo-johnson", standardize=False)

num_transformed = pt.fit_transform(num_df)

# Convertir de nuevo a DataFrame
num_transformed_df = pd.DataFrame(
    num_transformed,
    columns=num_df.columns,
    index=df.index
)

num_transformed_df.head()


,id,visita,grupo,periodo,tiempo,tratamiento,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,...,horas_cama,pgh_por_eficiencia,pgh_eficiencia,pgh_perturbaciones,pgh_disfuncion_dia,pgh_duracion_sueño,pgh_total,cat_pgh_total,total_meds,menst_flag
0,0.892335,0.882548,0.361236,0.693147,0.693147,0.693147,2.29732,2.553456,21.517152,13.734653,...,1.000189,5.917392e+06,0.374578,0.842911,-0.00000,1.509477,1.805491,0.344374,0.254710,-0.0
1,0.892335,1.624517,0.361236,0.693147,1.098612,0.693147,2.29732,2.553456,21.517152,13.734653,...,1.000189,1.609402e+07,-0.000000,0.842911,-0.00000,0.000000,0.695196,0.344374,-0.000000,-0.0
2,0.892335,2.287780,0.361236,1.098612,0.693147,1.098612,2.29732,2.553456,21.517152,13.734653,...,0.995083,1.211892e+07,-0.000000,0.842911,0.66197,0.844014,1.620512,0.344374,-0.000000,-0.0
3,0.892335,2.898055,0.361236,1.098612,1.098612,1.098612,2.29732,2.553456,21.517152,13.734653,...,0.989685,1.584221e+07,-0.000000,0.842911,-0.00000,0.000000,0.695196,0.344374,-0.000000,-0.0
4,1.654381,0.882548,0.422716,0.693147,0.693147,1.098612,2.29732,1.359119,16.107403,13.734653,...,0.989685,7.696188e+06,-0.000000,0.842911,0.66197,1.509477,1.620512,0.344374,0.258138,-0.0


In [6]:
df_transformed = df.copy()

# Reemplazar solo columnas numéricas
df_transformed[num_df.columns] = num_transformed_df

print("Dataset transformado:", df_transformed.shape)
df_transformed.head()


Dataset transformado: (168, 524)


,id,visita,grupo,periodo,tiempo,tratamiento,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,...,pgh_por_eficiencia,pgh_eficiencia,pgh_perturbaciones,pgh_disfuncion_dia,pgh_duracion_sueño,pgh_total,cat_pgh_total,total_meds,menst_flag,menst_tipo
0,0.892335,0.882548,0.361236,0.693147,0.693147,0.693147,2.29732,2.553456,21.517152,13.734653,...,5.917392e+06,0.374578,0.842911,-0.00000,1.509477,1.805491,0.344374,0.254710,-0.0,MPM
1,0.892335,1.624517,0.361236,0.693147,1.098612,0.693147,2.29732,2.553456,21.517152,13.734653,...,1.609402e+07,-0.000000,0.842911,-0.00000,0.000000,0.695196,0.344374,-0.000000,-0.0,MPM
2,0.892335,2.287780,0.361236,1.098612,0.693147,1.098612,2.29732,2.553456,21.517152,13.734653,...,1.211892e+07,-0.000000,0.842911,0.66197,0.844014,1.620512,0.344374,-0.000000,-0.0,MPM
3,0.892335,2.898055,0.361236,1.098612,1.098612,1.098612,2.29732,2.553456,21.517152,13.734653,...,1.584221e+07,-0.000000,0.842911,-0.00000,0.000000,0.695196,0.344374,-0.000000,-0.0,MPM
4,1.654381,0.882548,0.422716,0.693147,0.693147,1.098612,2.29732,1.359119,16.107403,13.734653,...,7.696188e+06,-0.000000,0.842911,0.66197,1.509477,1.620512,0.344374,0.258138,-0.0,Hombre


In [7]:
col = "bdnf"   # Cambia por cualquier variable

print("Antes:")
print(df[col].describe())

print("\nDespués (Yeo–Johnson):")
print(df_transformed[col].describe())

Antes:
count    168.000000
mean      29.079133
std       24.411850
min        2.084670
25%       13.451190
50%       23.520976
75%       37.572670
max      179.433282
Name: bdnf, dtype: float64

Después (Yeo–Johnson):
count    168.000000
mean       3.509406
std        0.912645
min        1.169343
25%        2.920380
50%        3.562156
75%        4.130029
max        6.196449
Name: bdnf, dtype: float64


In [8]:
df_transformed.to_excel("MITOS_YeoJohnson.xlsx", index=False)
print("Archivo guardado correctamente.")

Archivo guardado correctamente.


En esta fase de ingeniería de características se generó una versión transformada del conjunto de datos utilizando la transformación de potencia Yeo–Johnson, aplicada a las variables numéricas.

El dataset incluía múltiples tipos de mediciones biomédicas y conductuales, muchas de las cuales presentan distribuciones altamente asimétricas:

* biomarcadores con colas largas
* variables de consumo con muchos ceros
* mediciones fisiológicas con valores extremos naturales

En estos casos, trabajar con los valores originales puede generar:

* alta influencia de outliers
* varianza inestable
* dificultad para aplicar modelos estadísticos interpretables

Por ello, la transformación busca mejorar la estructura estadística del dataset antes del modelado.